In [1]:
%load_ext autoreload
%autoreload 2

# mount drive
from google.colab import drive
drive.mount('/content/drive')

# cd into project directory
%cd /content/drive/My\ Drive/Georgia_Tech/Spring_2021/sbic_stereotypes/baselines

# Useful Constants
DATA_DIR = '../data/'

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1M873oJOlIb1Bd5vliq6d04b61XsHLT5h/sbic_stereotypes/baselines


In [2]:
!pip install transformers
!pip install datasets

import torch
import pandas as pd
import numpy as np

     |████████████████████████████████| 2.1MB 25.5MB/s 
     |████████████████████████████████| 870kB 48.7MB/s 
     |████████████████████████████████| 3.3MB 43.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=f37b625970e727c904cf5fd7f2170f873f7ec0ddea6f07aaf6b11ddf6527a4eb
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
     |████████████████████████████████| 194kB 18.2MB/s 
     |████████████████████████████████| 112kB 45.0MB/s 
     |████████████████████████████████| 245kB 20.5MB/s 


In [3]:
## Set all parameters here ##
gpt_dict_5epoch = {
                    'MODEL_NAME':'openai-gpt',
                    'OUTPUT_DIR':'model/gpt_5epoch',
                    'LEARNING_RATE': 5e-6,
                    'EPOCHS': 5.0,
                    'SEED': 635
                  }

gpt2_dict_5epoch =  {
                      'MODEL_NAME':'gpt2',
                      'OUTPUT_DIR':'model/gpt2_5epoch',
                      'LEARNING_RATE': 1e-5,
                      'EPOCHS': 5.0,
                      'SEED': 574
                    }

active_dict = gpt2_dict_5epoch

In [4]:
from training_utils import *

# Read and Clean Data
from_train_file = DATA_DIR + 'SBIC.v2.trn.csv'
from_dev_file = DATA_DIR + 'SBIC.v2.dev.csv'
to_train_file = 'data/baseline_train_text.csv'
to_dev_file = 'data/baseline_dev_text.csv'

clean_df(from_train_file, to_train_file)
clean_df(from_dev_file, to_dev_file)

In [5]:
from datasets import load_dataset

datasets = load_dataset("csv", data_files={"train": to_train_file, "validation": to_dev_file})
print(datasets)

Using custom data configuration default-ceea53db03a7a92e


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ceea53db03a7a92e/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 112900
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 16738
    })
})


In [6]:
# We need to create the model and tokenizer
tokenizer = setup_tokenizer(active_dict['MODEL_NAME'])
tokenized_datasets = datasets.map(lambda examples: tokenizer(examples["text"]), \
                                    batched=True, num_proc=4, \
                                    remove_columns=["text"])

print(tokenized_datasets)





DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 112900
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 16738
    })
})


In [7]:
# Normalize the text length as Blocks
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    
    if total_length % block_size:
      remainder = block_size - (total_length % block_size)
    else:
      remainder = 0
    
    pad_extension = [tokenizer.pad_token_id for _ in range(remainder)]
    attention_extension = [0 for _ in range(remainder)]

    concatenated_examples['input_ids'].extend(pad_extension)
    concatenated_examples['attention_mask'].extend(attention_extension)
    total_length += remainder
    
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

print(lm_datasets)





DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 39169
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 5847
    })
})


In [ ]:
# Load PreTrained Model and train it
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments, trainer_utils
import math

trainer_utils.set_seed(active_dict['SEED'])

model = AutoModelForCausalLM.from_pretrained(active_dict['MODEL_NAME'])
model.resize_token_embeddings(len(tokenizer))
model.train()

num_epochs = active_dict['EPOCHS']
num_rows = lm_datasets['train'].num_rows
batch_size = 4

if num_epochs == 1:
  warmup_steps = math.ceil(num_rows / batch_size) // 2
  save_steps = warmup_steps * 2
  eval_steps = (save_steps * 10.0) // 100
else:
  warmup_steps = math.ceil(num_rows / batch_size)
  save_steps = (warmup_steps * num_epochs) // 2
  eval_steps = (warmup_steps * num_epochs * 5.0) // 100

warmup_steps = 5000
print("Linear Warm Up: ", warmup_steps)
print("Save Steps: ", save_steps)
print("Eval Steps: ", eval_steps)

training_args = TrainingArguments(
    output_dir = active_dict['OUTPUT_DIR'],
    evaluation_strategy = 'steps',
    eval_steps = eval_steps,
    save_steps = save_steps,
    save_total_limit = 1,
    warmup_steps = warmup_steps,
    learning_rate = active_dict['LEARNING_RATE'],
    per_device_train_batch_size = batch_size,
    num_train_epochs = num_epochs,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

trainer.train()


Linear Warm Up:  5000
Save Steps:  24482.0
Eval Steps:  2448.0


Step,Training Loss,Validation Loss,Runtime,Samples Per Second
2448,4.182100,3.390189,73.471700,79.582000
4896,3.119500,2.875593,73.761200,79.269000
7344,2.932700,2.763374,73.482200,79.570000
9792,2.833600,2.711517,73.693400,79.342000
12240,2.770300,2.683781,73.588500,79.455000
